# Reparatietijd berekener door Koen, Guy en Ruben

## Module Imports

 (Niet geïnstalleerd? Doe in command prompt: pip install *module_naam* (bijvoorbeeld pip install sklearn)

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

## Data Voorbereiding

In [2]:
def data_reading():
    df = pd.read_csv('sap_storing_data_hu_project.csv', low_memory = False)
    df = df.drop_duplicates(subset=['#stm_sap_meldnr'])
    df = df.reset_index(drop=True)

    df['stm_aanntpl_tijd']= pd.to_datetime(df['stm_aanntpl_tijd'])
    df['stm_fh_ddt']= pd.to_datetime(df['stm_fh_ddt'])

    # Calculate the true reparation time
    df['stm_hers_tijd'] = (df['stm_fh_ddt'].dt.hour * 60 + df['stm_fh_ddt'].dt.minute) - (df['stm_aanntpl_tijd'].dt.hour * 60 + df['stm_aanntpl_tijd'].dt.minute)

    df = df.dropna(subset=['stm_hers_tijd']) 
    df = df.reset_index(drop=True)
    return df

def prep_data(df, nan_columns):    
    df = df.dropna(subset=nan_columns) #['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit']
    df = df.reset_index(drop=True)
    df = df[(df['stm_hers_tijd'] >= 4) & (df['stm_hers_tijd'] <= 480)]
    df = df.reset_index(drop=True)
    return df

## Model Gebruik

In [3]:
def dtr_train_model(df, dummies_columns):
    DummiesX = pd.get_dummies(data=df[dummies_columns]) # ['stm_oorz_code', 'stm_prioriteit']
    y = df['stm_hers_tijd']
    X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)
    DTR = DecisionTreeRegressor(max_depth=12, min_samples_leaf = 2)
    DTR.fit(X_train,y_train)
    return X_train, X_test, y_train, y_test, DTR

def total_data_observation(X_train, X_test, y_train, y_test):
    DTR = DecisionTreeRegressor(max_depth=12, min_samples_leaf = 2)
    DTR.fit(X_train,y_train)
    y_pred = DTR.predict(X_test)
    # probability begin
    nodes_y_train = pd.DataFrame({'node_id': DTR.apply(X_train), 'y': y_train})
    nodes_std = nodes_y_train.groupby('node_id')['y'].agg(['std'])
    y_proba = pd.DataFrame({'node_id': DTR.apply(X_test), 'node_rmse': 0})
    for i in range(0, len(y_proba)):
        y_proba['node_rmse'].iloc[i] = nodes_std['std'].loc[y_proba['node_id'].iloc[i]]
        if np.mod(i, 10000) == 0:
            print("\ngevonden RMSE Waardes: " + str(i)) 
    print("betrouwbaarheden per Node ID in RSME waardes:\n")
    print(y_proba.sort_values(by='node_rmse', ascending = True))
    # probility einde
    RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test, y_pred)
    return r2, RMSE

def one_input_tester(oorzaakcode, prioriteit, X_train, y_train, regr):
    def dataframe_creator(oorzaakcode, prioriteit):
        X_test = pd.DataFrame({'stm_oorz_code': [oorzaakcode], 'stm_prioriteit': [prioriteit]})
        return X_test
    
    def predictor(regr, X_train, y_train, X_test):
#         #probability begin:
        nodes_y_train = pd.DataFrame({'node_id': regr.apply(X_train), 'y': y_train})
        nodes_std = nodes_y_train.groupby('node_id')['y'].agg(['std'])
        y_proba = pd.DataFrame({'node_id': regr.apply(X_test), 'node_rmse': 0})
        for i in range(0, len(y_proba)):
            y_proba['node_rmse'].iloc[i] = nodes_std['std'].loc[y_proba['node_id'].iloc[i]]
        
        betrouwbaarheid = y_proba.sort_values(by='node_rmse', ascending = True)
        betrouwbaarheid = float(str(betrouwbaarheid)[35:-5])
# #         #probability einde                       
        y_pred = regr.predict(X_test)
        output = round(y_pred[0])

        return output, betrouwbaarheid
    X_test = dataframe_creator(oorzaakcode, prioriteit)
    output, betrouwbaarheid = predictor(regr, X_train, y_train, X_test)
    return output, betrouwbaarheid





Voer de volgende code 1x uit:

In [4]:
df = data_reading()
df_RFR = prep_data(df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']], ['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit'])


In [5]:
X_train, X_test, y_train, y_test, DTR = dtr_train_model (df_RFR, ['stm_oorz_code', 'stm_prioriteit'])
df_RFR

,#stm_sap_meldnr,stm_hers_tijd,stm_oorz_groep,stm_prioriteit,stm_oorz_code
0,50053257,62.0,ONR-DERD,9.0,142.0
1,50053290,40.0,ONR-DERD,9.0,142.0
2,50053292,214.0,ONR-DERD,9.0,145.0
3,50053317,425.0,WEER,9.0,189.0
4,50053337,405.0,ONR-DERD,9.0,142.0
...,...,...,...,...,...
344474,99003502,89.0,TECHONV,4.0,298.0
344475,99003503,179.0,TECHONV,4.0,221.0
344476,99003505,24.0,TECHONV,5.0,215.0
344477,99003506,36.0,TECHONV,2.0,218.0


Voer de volgende code om achter de R2 score en Root Mean Square Error te weten van de hele model:

In [6]:
r2, RMSE = total_data_observation(X_train, X_test, y_train, y_test)
print("\nR2 score geeft: {} & Totale Root Mean Square Error waarde geeft: {}".format(r2, RMSE))


gevonden RMSE Waardes: 0


d:\users\ruben\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



gevonden RMSE Waardes: 10000

gevonden RMSE Waardes: 20000

gevonden RMSE Waardes: 30000

gevonden RMSE Waardes: 40000

gevonden RMSE Waardes: 50000

gevonden RMSE Waardes: 60000

gevonden RMSE Waardes: 70000

gevonden RMSE Waardes: 80000
betrouwbaarheden per Node ID in RSME waardes:

       node_id   node_rmse
83455       71    7.595817
82948       71    7.595817
64678      154    8.292722
65651      154    8.292722
77963      154    8.292722
...        ...         ...
12634      532  226.718621
14443      532  226.718621
48470      532  226.718621
58044      180  238.979288
69258      180  238.979288

[86120 rows x 2 columns]

R2 score geeft: 0.07205396663970509 & Totale Root Mean Square Error waarde geeft: 74.75114908557924


Voer de volgende code om de model te testen met maar 2 inputs waardes:

In [7]:
oorzaakcode = 66
prioriteit = 8
reparatietijd, betrouwbaarheid = one_input_tester (oorzaakcode, prioriteit, X_train, y_train, DTR)
print("\nGeschatte reparatietijd geeft: {} & Betrouwbaarheid waarde geeft: {}".format(reparatietijd, round(betrouwbaarheid)))


Geschatte reparatietijd geeft: 100 & Betrouwbaarheid waarde geeft: 7


## Applicatie (check onderaan om alles te runnen)

In [8]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import *

In [9]:
def application():
    root = tk.Tk()

    root.title("ProRail applicatie HBO-AI 2020 groep 3")
    proRailLogoColour = '#CD1E44'
    backgroundColour = '#FFFFFF'

    root.configure(background=backgroundColour)

    def activateSendButton():
        global oorzaakCode2
        global prioriteit2
        sendButton.config(text="Bereken Hersteltijd voor {}, {}".format(oorzaakCode2, prioriteit2))
        if len(oorzaakCode2) != 0:
            if len(prioriteit2) != 0:
                sendButton.config(state=NORMAL)

    def oorzOnClick():
        global oorzaakCode2
        oorzaakCode2 = oorzEntry.get()
        oorzLabel.config(text="Oorzaakcode: {}".format(oorzaakCode2))
        activateSendButton()
        print(calculatedTime)
    
    

    def prioOnClick():
        global prioriteit2
        prioriteit2 = prioEntry.get()
        prioLabel.config(text="Prioriteit: {}".format(prioriteit2))
        activateSendButton()

    def sendInfo():
        sendButton.config(state=DISABLED)
        # Hier output aftappen !!!
        global oorzaakCode2
        global prioriteit2
        calculatedTime, betrouwbaarheid = one_input_tester (oorzaakCode2, prioriteit2, X_train, y_train, DTR)    
        herstel2Label.config(text='{} minuten'.format(calculatedTime))
        betr2Label.config(text='{}'.format(betrouwbaarheid))
        
        oorzaakCode2 = ""
        prioriteit2 = ""
        oorzLabel.config(text="Oorzaakcode: {}".format(oorzaakCode2))
        prioLabel.config(text="Prioriteit: {}".format(prioriteit2))
        sendButton.config(text="Bereken Hersteltijd")

    BlankFrame = Frame(master=root, bg=proRailLogoColour, width=800, height=10, pady=30)
    BlankFrame.pack(fill=BOTH)

    bgList = Frame(master=root, bg=backgroundColour, width=800)
    bgList.pack(fill=BOTH)

    meldnrFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10, padx=70)
    meldnrFrame.pack(fill=BOTH, side=TOP)

    meldnrLabel = Label(master=meldnrFrame, text="Meldingnummer:", 
                        foreground="black", background=proRailLogoColour, font='Bold, 20', anchor=W)
    meldnrLabel.pack(side=LEFT)

    meldnrEntry = Entry(master=meldnrFrame, width=20, justify=LEFT, relief=RAISED, font='Bold, 16')
    meldnrEntry.pack(side=LEFT)


    oorzFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    oorzFrame.pack(side=TOP)

    oorzLabel = Label(master=oorzFrame, text="Oorzaakcode: {}".format(oorzaakCode2), 
                        foreground="black", background=proRailLogoColour, font='Bold, 18', anchor=W)
    oorzLabel.pack(padx=30, anchor=NW, side=TOP)

    invoerFrame = Frame(master=oorzFrame, bg=proRailLogoColour, width=600, height=0)
    invoerFrame.pack(fill=BOTH, side=TOP)

    oorzEntry = Entry(master=invoerFrame, width=20, justify=LEFT, relief=RAISED, font='Bold, 16')
    oorzEntry.pack(fill=BOTH, side=LEFT, pady=10, padx=30)

    oorzButton = Button(master=invoerFrame, command=oorzOnClick, text="Accepteer", height=3, width=10)
    oorzButton.pack(fill=BOTH, side=RIGHT, pady=10, padx=10)



    prioFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    prioFrame.pack(side=TOP)

    prioLabel = Label(master=prioFrame, text="Prioriteit: {}".format(prioriteit2),
                        foreground="black", background=proRailLogoColour, font='Bold, 18', anchor=W)
    prioLabel.pack(padx=30, anchor=NW, side=TOP)

    invoerFrame2 = Frame(master=prioFrame, bg=proRailLogoColour, width=600, height=0)
    invoerFrame2.pack(fill=BOTH, side=TOP)

    prioEntry = Entry(master=invoerFrame2, width=20, justify=LEFT, relief=RAISED, font='Bold, 16')
    prioEntry.pack(fill=BOTH, side=LEFT, pady=10, padx=30)

    prioButton = Button(master=invoerFrame2, command=prioOnClick, text="Accepteer", height=3, width=10)
    prioButton.pack(fill=BOTH, side=RIGHT, pady=10, padx=10)



    sendFrame = Frame(master=bgList, bg=backgroundColour, width=800, height=0, pady=10)
    sendFrame.pack(side=TOP)

    sendButton = Button(master=sendFrame, command=sendInfo, text="Bereken Hersteltijd", height=3, state=DISABLED)
    sendButton.pack(fill=BOTH, pady=10, padx=20)



    # Reparatietijd/hersteltijd en betrouwbaarheid

    herstelFrame = Frame(master=bgList, bg=proRailLogoColour, width=800, height=0, pady=10)
    herstelFrame.pack(side=TOP)

    herstelLabel = Label(master=herstelFrame, text='Berekende hersteltijd:', font='Bold, 16', anchor=W, bg=proRailLogoColour)
    herstelLabel.pack(padx=30, side=TOP)

    herstel2Label = Label(master=herstelFrame, text='{}'.format(calculatedTime), font='Bold, 26', anchor=W)
    herstel2Label.pack(padx=30, pady=40, side=TOP)

    betrLabel = Label(master=herstelFrame, text='Betrouwbaarheid: \n (Dichter bij 0 is beter)', font='Bold, 16', anchor=W, bg=proRailLogoColour)
    betrLabel.pack(padx=30, side=TOP)

    betr2Label = Label(master=herstelFrame, text='{}'.format(betrouwbaarheid), font='Bold, 26', anchor=W)
    betr2Label.pack(padx=30, pady=40, side=TOP)



    root.mainloop()

#### Run beide 2 cellen om te runnen

In [10]:
oorzaakCode2 = ""
prioriteit2 = ""
calculatedTime = ""
betrouwbaarheid = ""

In [11]:
application()